# Redaction and Prediction

## calculating predictions for each redaction -variation 1

In [ ]:

def calculate_predictions_for_each_redaction(model_in_use,segmentwise_attributions,image,im,topk,top10list,prediction_id,list_of_masks,new_segs,predictions):    
    top_heatmaps=100
    masking_type='normal/'
    dict1={}
    dict2={}
    dict_for_redacted_images=[]
    all_predictions_after_this_redaction=[]
    for i in top10list:
        dict1[i]=[]
        dict2[i]=[]
        dict1[i].append(predictions[0][i].numpy())
        # dict2[i].append(100.0)
    numbered_masks = {ind: mask for ind, mask in enumerate(new_segs)}
    oldmask=np.zeros(numbered_masks[0].shape, dtype=bool)
    all_predictions_after_this_redaction.append([])
    # for i in top10list:
    #         dict1[i].append(predictions1[0][i].numpy())
    

    # while top_heatmaps>=0:
    #     # for top_heatmaps in range(100,-0.9,-0.1)
    #     mask = segmentwise_attributions[prediction_id] > np.percentile(segmentwise_attributions[prediction_id], top_heatmaps)
    #     new_im=mask_on_image(im , mask, masking_type)
    #     new_im1=mask_on_image(image , mask, masking_type)
    #     (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
    #     dict_for_redacted_images.append(new_im1)
    #     for i in top10list:
    #         dict1[i].append(predictions1[0][i].numpy())
    #     top_heatmaps=top_heatmaps-1

    for mask_id in list_of_masks[prediction_id]:
        # for top_heatmaps in range(100,-0.9,-0.1)
        mask=numbered_masks[mask_id]
        mask=np.logical_or(mask,oldmask)
        new_im=mask_on_image(im , mask, masking_type)
        new_im1=mask_on_image(image , mask, masking_type)
        oldmask=mask
        # plt.imshow(mask)
        # plt.show()
        
        (predictions1,decoded_predictions1,top10list1,top1000list1,id_to_name1)=predict_img(model_in_use,new_im,print_data=False)
        dict_for_redacted_images.append(new_im1)
        all_predictions_after_this_redaction.append(predictions1[0].numpy())
        for i in top10list:
            dict1[i].append(predictions1[0][i].numpy())

    # plt.imshow(mask)
    # plt.show()
    
    for key in dict1:
        for k in range(len(dict1[key])):                
            dict2[key].append((dict1[key][k]/dict1[key][0])*100)
    return [dict1,dict2,dict_for_redacted_images,all_predictions_after_this_redaction]




In [ ]:

def load_predictions_by_redactions(model_in_use,segmentwise_attributions,image,im,topk,top10list,pth,list_of_masks,new_segs,predictions,redo=False,dont_save=False):

    if os.path.exists(pth+'_redactions'+'.pkl') and redo==False:
        with open(pth+'_redactions'+'.pkl', 'rb') as file:
            x=pickle.load(file)
            # [final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images]=pickle.load(file)
        print('loaded final predictions after redactions from file for type1 check')
        return x
        # return final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images,all_predictions_after_redaction

    else:
        all_predictions_after_redaction={}
        final_prediction_dictionary={}
        final_prediction_dictionary_with_percentages={}
        final_redacted_images={}
        for prediction_id in top10list[:topk]:
            print('loading predictions while redating '+str(prediction_id))
            prediction_dictionary,prediction_dictionary_with_percentages,dict_for_redacted_images,all_predictions_after_this_redaction=calculate_predictions_for_each_redaction(model_in_use,segmentwise_attributions,image,im,topk,top10list,prediction_id,list_of_masks,new_segs,predictions)
            final_prediction_dictionary[prediction_id]=prediction_dictionary
            final_prediction_dictionary_with_percentages[prediction_id]=prediction_dictionary_with_percentages
            all_predictions_after_redaction[prediction_id]=all_predictions_after_this_redaction
            # final_redacted_images[prediction_id]=dict_for_redacted_images # undo this if you are not using function to calcluate redaction images

        if dont_save== False:
            with open(pth+'_redactions'+'.pkl', 'wb') as file:
                pickle.dump([final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images,all_predictions_after_redaction], file)
                file.close()

        return [final_prediction_dictionary,final_prediction_dictionary_with_percentages,final_redacted_images,all_predictions_after_redaction]


## plotting

In [7]:

def plot_for_type1_redaction1(redacted_image_with_pred1,redacted_mask_with_pred1,final_prediction_dictionary,first_redaction,id_to_name,prediction1,prediction2,redaction1_percentile,type_of_graph):
    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred1)
    axs.flat[1].imshow(redacted_mask_with_pred1)
    
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()

    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    # for xy in list(final_prediction_dictionary[prediction1].keys())[:5]:
    #     axs.flat[0].plot(range(len(final_prediction_dictionary[prediction1][xy])),final_prediction_dictionary[prediction1][xy],label=id_to_name[xy])
    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction1][prediction1])),final_prediction_dictionary[prediction1][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction1][prediction2])),final_prediction_dictionary[prediction1][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    if type_of_graph == 'prob':
        # axs.flat[0].set_title('probabilities vs redactions')
        axs.flat[0].set_ylabel('probabilities',fontsize=('12'))
        axs.flat[0].set_xlabel('redactions',fontsize=('12'))
    elif type_of_graph == 'pre_sm':
        axs.flat[0].set_title('pre_softmax vs redactions')
        axs.flat[0].set_ylabel('pre_softmax')
        axs.flat[0].set_xlabel('redactions')
    axs.flat[0].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    # for xy in list(first_redaction.keys())[:5]:
    #     axs.flat[1].plot(range(len(first_redaction[xy])),first_redaction[xy],label=id_to_name[xy])
    axs.flat[1].plot(range(len(first_redaction[prediction1])),first_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(first_redaction[prediction2])),first_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    # axs.flat[1].set_title('percentages vs redactions')
    axs.flat[1].set_ylabel('Percentages',fontsize=('12'))
    axs.flat[1].set_xlabel('Number of redactions',fontsize=('12'))
    axs.flat[1].axvline(x=redaction1_percentile, color='r', linestyle=':',label=str(redaction1_percentile))
    plt.savefig('spider5/type2_graph1_fig.svg',format='svg')
    plt.show()

def plot_for_type1_redaction2(redacted_image_with_pred2,redacted_mask_with_pred2,final_prediction_dictionary,second_redaction,id_to_name,prediction1,prediction2,redaction2_percentile):


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    axs.flat[0].imshow(redacted_image_with_pred2)
    axs.flat[1].imshow(redacted_mask_with_pred2)
    axs.flat[0].axis('off')
    axs.flat[1].axis('off')
    plt.show()


    ROWS=1
    COLS=2
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-8)))
    # for xy in list(final_prediction_dictionary[prediction2].keys())[:5]:
    #     axs.flat[0].plot(range(len(final_prediction_dictionary[prediction2][xy])),final_prediction_dictionary[prediction2][xy],label=id_to_name[xy])

    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction2][prediction1])),final_prediction_dictionary[prediction2][prediction1],label=id_to_name[prediction1])
    axs.flat[0].plot(range(len(final_prediction_dictionary[prediction2][prediction2])),final_prediction_dictionary[prediction2][prediction2],label=id_to_name[prediction2])
    axs.flat[0].legend()
    axs.flat[0].set_title('probabilities vs redaction percentile')
    axs.flat[0].set_ylabel('probabilities')
    axs.flat[0].set_xlabel('redaction percentile')
    axs.flat[0].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    # for xy in list(second_redaction.keys())[:5]:
    #     axs.flat[1].plot(range(len(second_redaction[xy])),second_redaction[xy],label=id_to_name[xy])

    axs.flat[1].plot(range(len(second_redaction[prediction1])),second_redaction[prediction1],label=id_to_name[prediction1])
    axs.flat[1].plot(range(len(second_redaction[prediction2])),second_redaction[prediction2],label=id_to_name[prediction2])
    axs.flat[1].legend()
    # axs.flat[1].set_title('percentages vs redaction percentile')
    axs.flat[1].set_ylabel('Percentages',fontsize=('12'))
    axs.flat[1].set_xlabel('Number of redactions',fontsize=('12'))
    axs.flat[1].axvline(x=redaction2_percentile, color='r', linestyle=':',label=str(redaction2_percentile))
    plt.savefig('spider5/type2_graph2_fig.svg',format='svg')
    plt.show()


def plot_for_type2_sequence_of_masks(segmentwise_attributions,prediction1,prediction2,redaction1_percentile,redaction2_percentile):
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction1_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
   
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()
    
    xy=segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100)
    mask_temp=np.zeros(xy.shape)
    mask_temp_array=[]
    mask_temp_array2=[]
    for o in range(redaction2_percentile+1):
        mask_temp_new = segmentwise_attributions[prediction2] > np.percentile(segmentwise_attributions[prediction2], 100-o)
        if np.all(mask_temp_new== mask_temp):
            continue
        else:
            mask_temp= mask_temp_new
            mask_temp_array.append(mask_temp_new)
            mask_temp_array2.append(o)
    
    
    n=len(mask_temp_array)                
    # n=redaction1_percentile+1
    ROWS = math.floor(math.sqrt(n))
    COLS = math.ceil(n / ROWS)
    UPSCALE_FACTOR = 10
    fig, axs = plt.subplots(ROWS, COLS, figsize=(ROWS * UPSCALE_FACTOR, COLS * (UPSCALE_FACTOR-5)))
    for o in range(len(mask_temp_array)):
        axs.flat[o].imshow(mask_temp_array[o])
        axs.flat[o].set_title(mask_temp_array2[o])
    # for o in range(redaction1_percentile+1):
    #     mask_temp = segmentwise_attributions[prediction1] > np.percentile(segmentwise_attributions[prediction1], 100-o)
    #     axs.flat[o].imshow(mask_temp)
    plt.show()

    
